In [11]:
import cv2
import numpy as np
import os


In [12]:
def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

In [13]:
def knn(train,test,k=5):
    l=[]
    m = train.shape[0]
    for i in range(m):
        ix = train[i,:-1]
        iy = train[i,-1]
        d = dist(test,ix)
        l.append([d,iy])
    l = sorted(l,key = lambda x:x[0])
    l = l[:k]
    l = np.array(l)[:,-1]
    
    new_val = np.unique(l,return_counts = True)
   
    a = np.argmax(new_val[1])
    return new_val[0][a]
        

In [14]:
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier("./haarcascade_frontalface_alt.xml")

face_data = []
labels = []

class_id = 0
names = {}

datapath = "./"

for fx in os.listdir(datapath):
    if(fx.endswith('.npy')):
        names[class_id] = fx[:-4]
        print("Loaded"+fx)
        data_item = np.load(datapath+fx)
        face_data.append(data_item)
        
        l = class_id * np.ones((data_item.shape[0],))
        labels.append(l)
        class_id += 1
        
face_dataset = np.concatenate(face_data,axis = 0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis = 1)
print(trainset.shape)


while True:
    ret,frame = cap.read()
    if ret == False:
        continue
    #cv2.imshow("frame",frame)
    
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray_frame,1.3,5)
    
    if len(faces)==0:
        continue
    
    for f in faces[-1:]:
        x,y,w,h = f
        offset = 10
        face_section = gray_frame[y-offset:y+h-offset,x-offset:x+w-offset]
        face_section = cv2.resize(face_section,(100,100))
        out = knn(trainset,face_section.flatten())
        pred = names[out]
        cv2.putText(gray_frame,pred,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(gray_frame,(x,y),(x+w,y+h),(0,255,255),2)
        
    
    cv2.imshow("gray_frame",gray_frame)
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

Loadedvarsha.npy
(26, 10000)
(26, 1)
(26, 10001)
